<a href="https://colab.research.google.com/github/leslie-zi-pan/kaggle-courses/blob/main/ryanluoli2_notebooks/Simple_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Logistic Regression on Bankruptcy Data

https://www.kaggle.com/competitions/spaceship-titanic/overview


# Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Kaggle data setup

### Authorise with kaggle json api key

In [ ]:
! pip install kaggle
! rm -rf ~/.kaggle
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Download kaggle dataset

In [ ]:
!kaggle competitions download -c bankruptcy-risk-prediction
# !kaggle datasets download -d dansbecker/melbourne-housing-snapshot
!unzip bankruptcy-risk-prediction

  0% 0.00/22.3k [00:00<?, ?B/s]
100% 22.3k/22.3k [00:00<00:00, 30.8MB/s]
Archive:  bankruptcy-risk-prediction.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


## Imports

In [ ]:
import os
import numpy as np
import pandas as pd

import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
print('Setup Complete')

Setup Complete


## Load the data

In [ ]:
# Path of the file to read
train_path = '/content/train.csv'
test_path = '/content/test.csv'
submission_path = '/content/submission_example.csv'

# Read the file into a variable
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
submission = pd.read_csv(submission_path)

## Examine data

In [ ]:
df_train.head(10)

,id,sum,term,payment,guarantees,reason,credits,other_credits,credit_report,marital_status,...,qualification,immigrant,residence_since,accommodation,estate,savings,dependents,phone,status,bankruptcy
0,0,1169,6,4,none,television or radio,2,none,critical account or other credits existing(not...,male single,...,skilled employee,yes,4,own,real estate,unknown or no savings account,1,yes,less-than 0 cu,0
1,1,5951,48,2,none,television or radio,1,none,existing credits paid back duly till now,female divorced or separated or married,...,skilled employee,yes,2,own,real estate,less-than 100 cu,1,none,0 to 200 cu,1
2,2,2096,12,2,none,education,1,none,critical account or other credits existing(not...,male single,...,unskilled resident,yes,3,own,real estate,less-than 100 cu,2,none,no checking account,0
3,3,7882,42,2,guarantor,furniture or equipment,1,none,existing credits paid back duly till now,male single,...,skilled employee,yes,4,for free,building society savings agreement or life ins...,less-than 100 cu,2,none,less-than 0 cu,0
4,4,4870,24,3,none,new car,2,none,delay in paying off in the past,male single,...,skilled employee,yes,4,for free,unknown or no property,less-than 100 cu,2,none,less-than 0 cu,1
5,5,9055,36,2,none,education,1,none,existing credits paid back duly till now,male single,...,unskilled resident,yes,4,for free,unknown or no property,unknown or no savings account,2,yes,no checking account,0
6,6,2835,24,3,none,furniture or equipment,1,none,existing credits paid back duly till now,male single,...,skilled employee,yes,4,own,building society savings agreement or life ins...,500 to 1000 cu,1,none,no checking account,0
7,7,6948,36,2,none,used car,1,none,existing credits paid back duly till now,male single,...,management or self-employed or highly qualifie...,yes,2,rent,car or other,less-than 100 cu,1,yes,0 to 200 cu,0
8,8,3059,12,2,none,television or radio,1,none,existing credits paid back duly till now,male divorced or separated,...,unskilled resident,yes,4,own,real estate,greater-than 1000 cu,1,none,no checking account,0
9,9,5234,30,4,none,new car,2,none,critical account or other credits existing(not...,male married or widowed,...,management or self-employed or highly qualifie...,yes,2,own,car or other,less-than 100 cu,1,none,0 to 200 cu,1


In [ ]:
# Print last rows using tail() function
df_train.tail(10)

,id,sum,term,payment,guarantees,reason,credits,other_credits,credit_report,marital_status,...,qualification,immigrant,residence_since,accommodation,estate,savings,dependents,phone,status,bankruptcy
790,790,1188,21,2,none,business,1,none,existing credits paid back duly till now,female divorced or separated or married,...,skilled employee,yes,4,own,building society savings agreement or life ins...,less-than 100 cu,2,none,0 to 200 cu,1
791,791,6313,24,3,none,used car,1,none,existing credits paid back duly till now,male single,...,management or self-employed or highly qualifie...,yes,4,own,car or other,unknown or no savings account,2,yes,no checking account,0
792,792,1221,6,1,none,furniture or equipment,2,none,critical account or other credits existing(not...,male married or widowed,...,skilled employee,yes,2,own,building society savings agreement or life ins...,unknown or no savings account,1,none,no checking account,0
793,793,2892,24,3,none,furniture or equipment,1,none,existing credits paid back duly till now,male divorced or separated,...,skilled employee,yes,4,for free,unknown or no property,less-than 100 cu,1,none,greater-than 200 cu or salary assignments for ...,0
794,794,3062,24,4,none,furniture or equipment,1,none,existing credits paid back duly till now,male single,...,skilled employee,yes,3,rent,unknown or no property,500 to 1000 cu,1,yes,no checking account,0
795,795,2301,9,2,none,furniture or equipment,1,none,existing credits paid back duly till now,female divorced or separated or married,...,skilled employee,yes,4,rent,building society savings agreement or life ins...,100 to 500 cu,1,none,no checking account,0
796,796,7511,18,1,none,used car,1,none,existing credits paid back duly till now,male single,...,skilled employee,yes,4,for free,building society savings agreement or life ins...,unknown or no savings account,2,yes,less-than 0 cu,1
797,797,1258,12,2,none,furniture or equipment,2,none,critical account or other credits existing(not...,female divorced or separated or married,...,unskilled resident,yes,4,rent,building society savings agreement or life ins...,less-than 100 cu,1,none,no checking account,0
798,798,717,24,4,none,new car,2,none,delay in paying off in the past,male married or widowed,...,skilled employee,yes,4,own,car or other,unknown or no savings account,1,yes,no checking account,0
799,799,1549,9,4,none,new car,1,none,existing credits paid back duly till now,male single,...,unemployed or unskilled non-resident,yes,2,own,real estate,unknown or no savings account,1,none,0 to 200 cu,0


In [ ]:
df_train.describe()

,id,sum,term,payment,credits,age,residence_since,dependents,bankruptcy
count,800.0000,800.000000,800.00000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000
mean,399.5000,3190.976250,20.65125,2.966250,1.396250,35.406250,2.841250,1.147500,0.298750
std,231.0844,2732.671789,12.15635,1.128806,0.569773,11.470317,1.106833,0.354825,0.457996
min,0.0000,250.000000,4.00000,1.000000,1.000000,19.000000,1.000000,1.000000,0.000000
25%,199.7500,1342.250000,12.00000,2.000000,1.000000,27.000000,2.000000,1.000000,0.000000
50%,399.5000,2281.500000,18.00000,3.000000,1.000000,33.000000,3.000000,1.000000,0.000000
75%,599.2500,3914.250000,24.00000,4.000000,2.000000,41.000000,4.000000,1.000000,1.000000
max,799.0000,15945.000000,72.00000,4.000000,4.000000,75.000000,4.000000,2.000000,1.000000


In [ ]:
df_train.columns

Index(['id', 'sum', 'term', 'payment', 'guarantees', 'reason', 'credits',
       'other_credits', 'credit_report', 'marital_status', 'age', 'employment',
       'qualification', 'immigrant', 'residence_since', 'accommodation',
       'estate', 'savings', 'dependents', 'phone', 'status', 'bankruptcy'],
      dtype='object')

## Data Preparation


1.   Drop the Id column
2.   Encode categorical variable with numerical values (aggregated percentage of bankruptcy)
3.   Standardize data

In [ ]:
df = pd.concat([df_train, df_test], axis=0, ignore_index=True)

In [ ]:
cat_features = df.columns[df.dtypes == 'object'].to_list()
cat_features.append('bankruptcy')

num_features = df.columns[df.dtypes != 'object'].to_list()
num_features.remove('bankruptcy')
num_features.remove('id')

In [ ]:
df1 = df.drop(['id'], axis=1).copy()

In [ ]:
# Encode all the categorical variables with aggregated percentage of bankruptcy
df2 = df1.copy()

for feature in cat_features[:-1]:
  df2[feature] = df2.groupby([feature])['bankruptcy'].transform(np.mean)

In [ ]:
# Standardize all the numerical features
df3 = df2.copy()
scaler = StandardScaler()
df3[df3.columns[:-1]] = scaler.fit_transform(df3[df3.columns[:-1]])

In [ ]:

X_train = df3.iloc[:df_train.shape[0],:-1].copy()
y_train = df3.iloc[:df_train.shape[0],-1].copy()

X_test = df3.iloc[df_train.shape[0]:,:-1].copy()

## Evalutation Metric

In [ ]:
def kf_cross_val(model, X, y):
  scores = []
  kf = KFold(n_splits=5, shuffle=True, random_state =52)
  for fold, (train_index, test_index) in enumerate(kf.split(X, y)):
    x_train = X.iloc[train_index]
    y_train = y.loc[train_index]
    x_test = X.loc[test_index]
    y_test = y.loc[test_index]

    model.fit(x_train, y_train)
    y_pred = model.predict_proba(x_test)[:, 1]

    scores.append(roc_auc_score(y_test, y_pred))

  return scores

## Logistic Regression

In [ ]:
lr = LogisticRegression(max_iter=2000, random_state=0)

lr_param = {'C': np.logspace(-10, 10)}

lr_cv = GridSearchCV(estimator=lr, param_grid=lr_param, scoring='roc_auc', cv=5)
lr_cv.fit(X_train, y_train)
lr_cv.best_params_

{'C': 0.0022229964825261957}

In [ ]:
lr_best = LogisticRegression(C = lr_cv.best_params_['C'], 
                             max_iter = 2000, 
                             random_state = 0)

lr_best.fit(X_train, y_train)

LogisticRegression(C=0.0022229964825261957, max_iter=2000, random_state=0)

In [ ]:
scores = kf_cross_val(lr_best, X_train, y_train)
round(np.mean(scores), 5)

0.81205

In [ ]:
lr_best.fit(X_train, y_train)
y_pred = lr_best.predict_proba(X_test)[:,1]

submission['proba'] = y_pred
submission.to_csv('submission.csv', index=False)
submission.head()

,id,proba
0,800,0.291856
1,801,0.254246
2,802,0.313673
3,803,0.164785
4,804,0.359552
